## 1. Импорт  библиотек

In [2]:
import random
import math
import copy

import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

import os
import tensorflow as tf
# import tensorflow_addons as tfa
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model

import torch
from kan import KAN

from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))
#%load_ext autotime

C:\Users\Gavriil\AppData\Local\Temp\ipykernel_29704\581288377.py:26: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [3]:
from tools import KAN_es
from algos import create_XY_data, alg_keras_mlp, vector_pred_NN, alg_KAN_model, vector_pred_KAN, multi_exp

## 3. Обучение нейронных сетей

### 3.0. Загрузка наборов данных

In [4]:
'''
initial_trn_data_fdfs = pd.read_csv("datasets_mtgm\mtgm_fdfs_10k_trn.csv")
initial_vld_data_fdfs = pd.read_csv("datasets_mtgm\mtgm_fdfs_10k_vld.csv")
initial_tst_data_fdfs = pd.read_csv("datasets_mtgm\mtgm_fdfs_10k_tst.csv")

initial_trn_data_udfs = pd.read_csv("datasets_mtgm\mtgm_udfs_10k_trn.csv")
initial_vld_data_udfs = pd.read_csv("datasets_mtgm\mtgm_udfs_10k_vld.csv")
initial_tst_data_udfs = pd.read_csv("datasets_mtgm\mtgm_udfs_10k_tst.csv")

initial_trn_data_udus = pd.read_csv("datasets_mtgm\mtgm_udus_10k_trn.csv")
initial_vld_data_udus = pd.read_csv("datasets_mtgm\mtgm_udus_10k_vld.csv")
initial_tst_data_udus = pd.read_csv("datasets_mtgm\mtgm_udus_10k_tst.csv")
'''
# !Some troubles with *.csv naming!

initial_trn_data_fdfs = pd.read_csv("datasets_mtgm\mtgm_fdfs_10k_trn.csv")
initial_vld_data_fdfs = pd.read_csv("datasets_mtgm\mtgm_fdfs_10k_tst.csv")
initial_tst_data_fdfs = pd.read_csv("datasets_mtgm\mtgm_fdfs_10k_pro.csv")

initial_trn_data_udfs = pd.read_csv("datasets_mtgm\mtgm_udfs_10k_trn.csv")
initial_vld_data_udfs = pd.read_csv("datasets_mtgm\mtgm_udfs_10k_tst.csv")
initial_tst_data_udfs = pd.read_csv("datasets_mtgm\mtgm_udfs_10k_pro.csv")

initial_trn_data_udus = pd.read_csv("datasets_mtgm\mtgm_udus_10k_trn.csv")
initial_vld_data_udus = pd.read_csv("datasets_mtgm\mtgm_udus_10k_tst.csv")
initial_tst_data_udus = pd.read_csv("datasets_mtgm\mtgm_udus_10k_pro.csv")

In [5]:
data_col_names = initial_trn_data_udus.columns

### 3.1. Шкалирование данных

In [6]:
mm_scaler = MinMaxScaler()
mm_scaler.fit(pd.concat([initial_trn_data_fdfs,
                         initial_trn_data_udfs, 
                         initial_trn_data_udus],
                        axis=0, sort=False, ignore_index=True))

MinMaxScaler()

In [7]:
'''
scaled_trn_data_fdfs = pd.DataFrame(mm_scaler.transform(initial_trn_data_fdfs),columns=data_col_names) 
scaled_vld_data_fdfs = pd.DataFrame(mm_scaler.transform(initial_vld_data_fdfs),columns=data_col_names)
scaled_tst_data_fdfs = pd.DataFrame(mm_scaler.transform(initial_tst_data_fdfs),columns=data_col_names)

scaled_trn_data_udfs = pd.DataFrame(mm_scaler.transform(initial_trn_data_udfs),columns=data_col_names)
scaled_vld_data_udfs = pd.DataFrame(mm_scaler.transform(initial_vld_data_udfs),columns=data_col_names)
scaled_tst_data_udfs = pd.DataFrame(mm_scaler.transform(initial_tst_data_udfs),columns=data_col_names)

scaled_trn_data_udus = pd.DataFrame(mm_scaler.transform(initial_trn_data_udus),columns=data_col_names)
scaled_vld_data_udus = pd.DataFrame(mm_scaler.transform(initial_vld_data_udus),columns=data_col_names)
scaled_tst_data_udus = pd.DataFrame(mm_scaler.transform(initial_tst_data_udus),columns=data_col_names)
'''
trn = pd.DataFrame(mm_scaler.transform(initial_trn_data_fdfs),columns=data_col_names) 
vld = pd.DataFrame(mm_scaler.transform(initial_vld_data_fdfs),columns=data_col_names)
tst = pd.DataFrame(mm_scaler.transform(initial_tst_data_fdfs),columns=data_col_names)


### Мульти-эксперимент

In [8]:
l_algos_names=['GMT_KAN_hn_1', 'GMT_KAN_hn_2', 'GMT_KAN_hn_5', 'GMT_KAN_hn_10']

l_algos=[vector_pred_KAN, vector_pred_KAN, vector_pred_KAN, vector_pred_KAN]

Data = (trn, vld, tst)

mult_data = [Data for i in range(4)]

l_geophysical_method = ['G' for i in range(4)]

output_parameter = ['H1_8', 'H2_8', 'H3_8']
l_output_parameter = [output_parameter for i in range(4)]

l_kwargs=[{},
          {},
          {},
          {}
          ]

l_metrics_names=['mae', 'rmse', 'r2']

num_iter=5

In [9]:
full_df = multi_exp(l_algos_names=l_algos_names,
                    l_algos=l_algos,
                    mult_data=mult_data,
                    l_geophysical_method=l_geophysical_method,
                    l_output_parameter=l_output_parameter,
                    l_kwargs=l_kwargs,
                    l_metrics_names=l_metrics_names,
                    num_iter=num_iter)

--- Processing G_MLP
iter: 1
Start train model: ?_G_all_H1_8_rs1
Number of epochs for training NN model:1017
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Start train model: ?_G_all_H2_8_rs1


In [1]:
full_df

NameError: name 'full_df' is not defined

In [ ]:
full_df.to_excel('full_metrics_3.xlsx')
#pd.read_excel('full_metrics.xlsx').drop('Unnamed: 0', axis=1)

In [ ]:
aggr_df = full_df.groupby(['alg_name']).agg(["mean", "std"]).drop(['iter'], axis=1)
aggr_df.to_excel('aggr_metrics_3.xlsx')
aggr_df